In [25]:
#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Initialise weights and bias matrices

The neural network will consist of the following layers:

Input layer = 3 nodes 


Hidden layer = 2 nodes


Output layer = 1 node 

Therefore, the weight and bias matrices must be initialised to suit the dimensions of each layer. 

The input data has three features: x1, x2, x3. To keep things simple, we wil work with one data sample. The input matrix is therefore a 1 x 3 matrix (features as columns). 

The weight matrix for the first layer will feed the input data (1 x 3 matrix) to the hidden layer (2 nodes). Therefore, the dimensions of the weight matrix will be 3 x 2 matrix. 3 rows for each feature, and 2 columns - one for each node in the hidden layer. 

The bias matrix will be a 1 x 2 matrix - one bias for each node in the hidden layer (2 nodes).  


In [3]:
# Example input - n = 1
inp = [np.array([1,3,4])]
out = np.array([[1]])
inp = np.vstack(inp)

# input layer dimensions have to be equal to the number of features from the input data set 
sizes = [3,2,1]
num_layers = len(sizes)

# initialise weights and biases to random values
biases = [np.random.randn(1,y) for y in sizes[1:]]
weights = [np.random.randn(x,y) for x,y in zip(sizes[:-1], sizes[1:])]
layers = len(sizes)

print("layers: ", num_layers)
print("weights: ", weights[0].shape)
print("bias: ", biases[0].shape)
print("inputs: ", inp.shape)
print("outputs: ", out.shape)

print("weights: ", weights)

layers:  3
weights:  (3, 2)
bias:  (1, 2)
inputs:  (1, 3)
outputs:  (1, 1)
weights:  [array([[ 0.67408638,  0.62548787],
       [ 0.44157195, -0.12582172],
       [ 0.44210482, -0.33442851]]), array([[-0.67978993],
       [-1.14167227]])]


In [4]:
def feedforward(a, w, b):
    z = np.dot(a,w) + b 
    anew = 1 / (1 + np.exp(z))
    return anew

def sigmoid(z):
    a = 1 / (1 + np.exp(z))
    return a

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def cost_derivative(a, y):
    e = a - y 
    return e 

# initialise inputs 
a = inp
activations = [a]
zs = []
nabla_b = [np.zeros(b.shape) for b in biases]
nabla_w = [np.zeros(w.shape) for w in weights]
print("nabla_w: ", nabla_w)

# feedforward 
# for each sample, feedforward through the layers - results in 1 outputs 
for w, b in zip(weights, biases):
    print("activations: ", a)
    z = np.dot(a, w) + b
    zs.append(z)
    a = sigmoid(z)
    print("Updated activations: ", a)
    activations.append(a)

print("final activations: ", activations)
print("final zs: ", zs)

nabla_w:  [array([[0., 0.],
       [0., 0.],
       [0., 0.]]), array([[0.],
       [0.]])]
activations:  [[1 3 4]]
Updated activations:  [[0.02435395 0.94804618]]
activations:  [[0.02435395 0.94804618]]
Updated activations:  [[0.6263032]]
final activations:  [array([[1, 3, 4]]), array([[0.02435395, 0.94804618]]), array([[0.6263032]])]
final zs:  [array([[ 3.69040597, -2.90404798]]), array([[-0.51638981]])]


In [13]:
# backward propagation 
# out = output defined in the first cell 
print("activations list: ", activations)
print("z list: ", zs)
for layer in range(len(sizes) - 1):
    print("layer: %s, activations: %s" % (layer + 1, activations[layer]))

# 1. Error of the output activations 
print("z: ", zs[-1])
print("out: ", out)

# calculate the error of the output layer to initiate backpropagation
delta = cost_derivative(activations[-1], out) * sigmoid_prime(zs[-1])
print("delta error: ", delta)

# 2. Error of layer 3 output activations wrto the layer 2 bias 
nabla_b[-1] = delta 
print("bias error: ", nabla_b[-1])

# 3. Error of layer 3 output activations wrto the layer 2 weights 
# error wrto weights in layer L is calculated by getting the activations of layer L-1 
# which is treated as the input to weights in layer L (Ain*weights = error out)
# print("activations from layer -2 (layer 1) ", activations[-2])
# print("transpose: ", activations[-2].transpose())
nb_weight = np.dot(activations[-2].transpose(), delta)
nabla_w[-1] = nb_weight
print("weight error: ", nb_weight)

# 4. Error of the layer 2 activations wrto to the layer 1 (input layer) bias
delta_2 = np.dot(delta, weights[-1].transpose()) * sigmoid_prime(zs[-2])
nabla_b[-2] = delta_2
print("bias error: ", nabla_b[-2])

# 5. Error of layer 2 activations wrto layer 1 weights
nabla_w[-2] = np.dot(activations[-3].transpose(), delta_2)
print("weight error: ", nabla_w[-2])




activations list:  [array([[1, 3, 4]]), array([[0.02435395, 0.94804618]]), array([[0.6263032]])]
z list:  [array([[ 3.69040597, -2.90404798]]), array([[-0.51638981]])]
layer: 1, activations: [[1 3 4]]
layer: 2, activations: [[0.02435395 0.94804618]]
z:  [[-0.51638981]]
out:  [[1]]
delta error:  [[-0.0874628]]
bias error:  [[-0.0874628]]
weight error:  [[-0.00213006]
 [-0.08291878]]
bias error:  [[0.00141273 0.00491826]]
weight error:  [[0.00141273 0.00491826]
 [0.0042382  0.01475479]
 [0.00565093 0.01967306]]


In [28]:
# 4. Adjust the weights and biases through gradient descent 
eta = 0.5 # learning rate 
# #delta_nabla_b, delta_nabla_w = self.backprop(x, y)

updated_weights = [w-(eta/len(inp))*nw for w, nw in zip(weights, nabla_w)]
updated_biases = [b-(eta/len(inp))*nb for b, nb in zip(biases, nabla_b)]

diff_weights = [new - old for new, old in zip(updated_weights, weights)]
diff_biases = [new - old for new, old in zip(updated_biases, biases)]
#diff_biases = updated_biases - biases

print("updated weights: \n", updated_weights, "\ndifference: \n", diff_weights)
print("updated biases: \n", updated_biases, "\ndifference: \n", diff_biases)



updated weights: 
 [array([[ 0.67338002,  0.62302873],
       [ 0.43945285, -0.13319912],
       [ 0.43927936, -0.34426504]]), array([[-0.6787249 ],
       [-1.10021288]])] 
difference: 
 [array([[-0.00070637, -0.00245913],
       [-0.0021191 , -0.0073774 ],
       [-0.00282546, -0.00983653]]), array([[0.00106503],
       [0.04145939]])]
updated biases: 
 [array([[-0.0775219 , -1.81681577]]), array([[0.62625519]])] 
difference: 
 [array([[-0.00070637, -0.00245913]]), array([[0.0437314]])]
